# This Notebook Answers Question 3 - Does the type of Complaints have obvious relationship with any particular characteristic(s) of the Houses ?

# <font color="maroon">Primary Land Use Tax Lot Output (PLUTO)</font>
<font size=4>Data file contains:
<li>Tax Lot, Building, and Geographic/Political/Administrative Districts information.</li>
</font>
<BR>
<font size=4>Borough Code:
<li>BX -- Bronx</li>
<li>BK -- Brooklyn</li>
<li>MN -- Manhattan</li>
<li>QN -- Queens</li>
<li>SI -- Staten Island</li>
</font>

In [1]:
!pip install --upgrade pip

Requirement already up-to-date: pip in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (18.1)


In [2]:
!pip install missingno

In [3]:
! pip install --upgrade matplotlib

Requirement already up-to-date: matplotlib in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (3.0.2)


In [4]:
!pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (0.20.2)


In [5]:
!pip install xgboost==0.80

In [6]:
!pip install imblearn

In [7]:
!pip install TextBlob

In [8]:
!pip install pandas-profiling 

In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import zipfile, requests, io, os, sys, types, time
import seaborn as sns
import matplotlib.cm as cm
import statsmodels.api as sm
import itertools
import matplotlib
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Binarizer
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'


from scipy import stats
from scipy.stats import norm, skew 

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import seaborn as sns


from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
#import lightgbm as lgb
from xgboost import XGBRegressor

from sklearn.preprocessing import RobustScaler, StandardScaler, MaxAbsScaler
from sklearn.metrics import mean_squared_error


from dateutil import parser, rrule
from datetime import datetime, time, date
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score

from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE

from IPython.display import display
%matplotlib inline

pd.options.display.float_format = '{:20,.3f}'.format
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 1000
np.set_printoptions(precision=3)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# <font color="blue" size="4">Next We shall download the building data, Pluto Data, that was stores in Cloud Object Store atthe end of lab of Chapter 1.</font>

In [10]:
# The code was removed by Watson Studio for sharing.

#### Since we ahave already identifiedthat Bronx has the highest heating complaints we shall only download the data for Bronx borough

In [11]:
client_4583a2827cdd4084aebc758e0b87cb12.download_file(Bucket=bucket,Key='pluto_bx_raw_cos.pkl',Filename='./pluto_bx_raw_local.pkl')

In [12]:
df_bx = pd.read_pickle('./pluto_bx_raw_local.pkl')
df_bx.shape

(89854, 87)

In [13]:
df_bx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89854 entries, 0 to 89853
Data columns (total 87 columns):
Borough                 89854 non-null object
Block                   89854 non-null int64
Lot                     89854 non-null int64
CD                      89854 non-null int64
CT2010                  89447 non-null float64
CB2010                  87342 non-null float64
SchoolDist              89695 non-null float64
Council                 89649 non-null float64
ZipCode                 89525 non-null float64
FireComp                89568 non-null object
PolicePrct              89692 non-null float64
HealthCenterDistrict    89727 non-null float64
HealthArea              89630 non-null float64
SanitBoro               89235 non-null float64
SanitDistrict           89238 non-null float64
SanitSub                88789 non-null object
Address                 89785 non-null object
ZoneDist1               89795 non-null object
ZoneDist2               2553 non-null object
ZoneDist3  

#### Let us select the columns as recommended in the instruction

In [14]:
df_bx = df_bx[['Address','BldgArea','BldgDepth','BuiltFAR',
                'CommFAR','FacilFAR', 
                'Lot','LotArea','LotDepth','NumBldgs','NumFloors',
                'OfficeArea','ResArea','ResidFAR','RetailArea',
                'YearBuilt','YearAlter1','ZipCode', 'YCoord', 'XCoord'
                ]]

In [15]:
df_bx.head(2)

,Address,BldgArea,BldgDepth,BuiltFAR,CommFAR,FacilFAR,Lot,LotArea,LotDepth,NumBldgs,NumFloors,OfficeArea,ResArea,ResidFAR,RetailArea,YearBuilt,YearAlter1,ZipCode,YCoord,XCoord
0,122 BRUCKNER BOULEVARD,0,0.000,0.000,5.000,6.500,1,15000,200.000,1,0.000,0,0,6.020,0,0,0,"10,454.000","232,162.000","1,005,957.000"
1,126 BRUCKNER BOULEVARD,752,16.000,0.050,5.000,6.500,4,13770,100.000,2,1.000,272,0,6.020,0,1931,1994,"10,454.000","232,156.000","1,006,076.000"


In [16]:
df_bx['ZipCode']              = df_bx['ZipCode'].fillna(0).astype(int)
df_bx['NumFloors']            = df_bx['NumFloors'].fillna(0).astype(int)
df_bx['NumBldgs']             = df_bx['NumBldgs'].fillna(0).astype(int)
df_bx['LotDepth']             = df_bx['LotDepth'].fillna(0).astype(int)
df_bx['BldgDepth']            = df_bx['BldgDepth'].fillna(0).astype(int)
df_bx['BldgArea']             = df_bx['BldgArea'].fillna(0).astype(int)
df_bx['Lot']                  = df_bx['Lot'].fillna(0).astype(int)
df_bx['LotArea']              = df_bx['LotArea'].fillna(0).astype(int)
df_bx['BuiltFAR']             = df_bx['BuiltFAR'].fillna(0).astype(int)
df_bx['ResidFAR']             = df_bx['ResidFAR'].fillna(0).astype(int)
df_bx['CommFAR']              = df_bx['CommFAR'].fillna(0).astype(int)
df_bx['FacilFAR']             = df_bx['FacilFAR'].fillna(0).astype(int)
df_bx['OfficeArea']           = df_bx['OfficeArea'].fillna(0).astype(int)
df_bx['ResArea']              = df_bx['ResArea'].fillna(0).astype(int)
df_bx['RetailArea']           = df_bx['RetailArea'].fillna(0).astype(int)
df_bx['XCoord']               = df_bx['XCoord'].fillna(0).astype(int)
df_bx['YCoord']               = df_bx['YCoord'].fillna(0).astype(int)

#### Next let us get the complaint data that we saved in Cloud Object Store at the end of previous Notebook

In [68]:
client_4583a2827cdd4084aebc758e0b87cb12.download_file(Bucket=bucket,Key='DataFromNoteBookQ2_cos.pkl',Filename='./DataFromNoteBookQ2_local.pkl')

In [69]:
df_complaint = pd.read_pickle('./DataFromNoteBookQ2_local.pkl')
df_complaint.shape

(520749, 19)

# <font color="maroon">Join PLUTO and Complaint dataset based on address</font>

# <font size="4" color="blue">Let's see how many addresses in complaints can be found in PLUTO dataset </font>

In [70]:
print(df_complaint.shape)
print(df_bx.shape)

(520749, 19)
(89854, 20)


In [71]:
perc=((df_complaint['incident_address'].isin(df_bx['Address']).sum())/df_complaint.shape[0])*100

print('Complaints incident address found in PLUTO dataset percentage: {:.3f}%'.format(perc))

Complaints incident address found in PLUTO dataset percentage: 80.682%


# <font color="blue" size="4">There is 80.682% Brooklyn complaint incident address can be found in Pluto dataset. (FYI. You may get a different number due to when the dataset was pulled from online.) </font>

#### Removing Duplicates

In [72]:
df_pluto_bx = df_bx[df_bx['Address'].notnull()]
print(df_pluto_bx.shape)
df_pluto_bx = df_pluto_bx.drop_duplicates(subset=['Address','ZipCode'], keep='first') #or last
print(df_pluto_bx.shape)

(89785, 20)
(87384, 20)


#### Next let us aggregate the number of Complaints at each address level so that we can try finding out if there is any relationship between the number of complaints and building characteristics

In [73]:
df_complaint_aggr = df_complaint.groupby(['incident_address'], as_index=False).agg({'unique_key':'count'})
df_complaint_aggr.columns = ["Address", "complaintCnt"]

In [74]:
df_complaint_aggr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21340 entries, 0 to 21339
Data columns (total 2 columns):
Address         21340 non-null object
complaintCnt    21340 non-null int64
dtypes: int64(1), object(1)
memory usage: 500.2+ KB


#### Now we merge the data between building dataset and aggregated complaint dataset

In [75]:
df_combo = pd.merge(df_bx, df_complaint_aggr, left_on=['Address'], right_on=['Address'], how='left')
print(df_combo.shape)
print(df_complaint_aggr.shape)
print(df_bx.shape)

(89854, 21)
(21340, 2)
(89854, 20)


In [76]:
df_combo.head(2)

,Address,BldgArea,BldgDepth,BuiltFAR,CommFAR,FacilFAR,Lot,LotArea,LotDepth,NumBldgs,NumFloors,OfficeArea,ResArea,ResidFAR,RetailArea,YearBuilt,YearAlter1,ZipCode,YCoord,XCoord,complaintCnt
0,122 BRUCKNER BOULEVARD,0,0,0,5,6,1,15000,200,1,0,0,0,6,0,0,0,10454,232162,1005957,nan
1,126 BRUCKNER BOULEVARD,752,16,0,5,6,4,13770,100,2,1,272,0,6,0,1931,1994,10454,232156,1006076,nan


In [77]:
df_combo['complaintCnt'] = df_combo.complaintCnt.fillna('0').astype(int)

#### Let us filter out the data that has YearBuilt as 0 to remove bad data

In [78]:
df_combo = df_combo[df_combo['YearBuilt'] > 0]
print(df_combo.shape)

(83487, 21)


#### Let us create 2 new features Building Age from YearBuilt and Builkding Altered Age from YearAlter1

In [79]:
df_combo['bldg_age'] = df_combo.apply(lambda x: 2018-x['YearBuilt'] , axis=1)
print(df_combo.shape)

(83487, 22)


In [80]:
df_combo['bldg_alt_age'] = df_combo.apply(lambda x: 2018-x['YearAlter1'] if x['YearAlter1']>0 else x['bldg_age'], axis=1)
print(df_combo.shape)

(83487, 23)


In [81]:
df_combo.head(2)

,Address,BldgArea,BldgDepth,BuiltFAR,CommFAR,FacilFAR,Lot,LotArea,LotDepth,NumBldgs,NumFloors,OfficeArea,ResArea,ResidFAR,RetailArea,YearBuilt,YearAlter1,ZipCode,YCoord,XCoord,complaintCnt,bldg_age,bldg_alt_age
1,126 BRUCKNER BOULEVARD,752,16,0,5,6,4,13770,100,2,1,272,0,6,0,1931,1994,10454,232156,1006076,0,87,24
2,138 BRUCKNER BOULEVARD,39375,200,1,5,6,10,35000,200,1,2,0,0,6,0,1931,0,10454,232036,1006187,0,87,87


#### Now let us divide the number of complaints to 4 ranges  - 
a) 0 (No Complaint), 
b) 1 (Number of Complaints between 1 to 10)
c) 10 (Number of Complaints between 11 to 100) and
d) 100 (Number of Complaints above 100)

In [82]:
def cnt_range(cnt):
    if cnt <= 0:
        rng = 0
    elif cnt <= 10 and cnt > 0:
        rng = 1
    elif cnt <= 100 and cnt > 10:
        rng = 10
    elif cnt > 100:
        rng = 100
    else:
        rng = 10
    return rng

In [83]:
df_combo['complaint_range'] = df_combo['complaintCnt'].apply(lambda x: cnt_range(x))
df_combo.shape

(83487, 24)

In [84]:
df_combo.groupby(['complaint_range']).count()

,Address,BldgArea,BldgDepth,BuiltFAR,CommFAR,FacilFAR,Lot,LotArea,LotDepth,NumBldgs,NumFloors,OfficeArea,ResArea,ResidFAR,RetailArea,YearBuilt,YearAlter1,ZipCode,YCoord,XCoord,complaintCnt,bldg_age,bldg_alt_age
complaint_range,,,,,,,,,,,,,,,,,,,,,,,
0,67154,67155,67155,67155,67155,67155,67155,67155,67155,67155,67155,67155,67155,67155,67155,67155,67155,67155,67155,67155,67155,67155,67155
1,10924,10924,10924,10924,10924,10924,10924,10924,10924,10924,10924,10924,10924,10924,10924,10924,10924,10924,10924,10924,10924,10924,10924
10,4483,4483,4483,4483,4483,4483,4483,4483,4483,4483,4483,4483,4483,4483,4483,4483,4483,4483,4483,4483,4483,4483,4483
100,925,925,925,925,925,925,925,925,925,925,925,925,925,925,925,925,925,925,925,925,925,925,925


#### The above table clearly shows that 80% of the addresses are not having any Heating Complaints and 10% of them have complaints less than 10 in last 8 years
#### So let us try to investigate if there is any relationship with number of complaints and the characteristic of the buildings 

In [38]:
df_combo_sub = df_combo[['Address','BldgArea','BldgDepth','BuiltFAR',
                'CommFAR','FacilFAR',
                'Lot','LotArea','LotDepth','NumBldgs','NumFloors',
                'OfficeArea','ResArea','ResidFAR','RetailArea',
                'bldg_age','bldg_alt_age','ZipCode', 'YCoord', 'XCoord',
                'complaintCnt', 'complaint_range']]
df_combo_sub.shape

(83487, 22)

#### Let us create a function to identify the correlation between various features

In [85]:
def order_corr(df,response): 
    df_new = df.corr()[[response]]   
    df_new = df_new.dropna()    
    df_new['abs'] = df_new[response].apply(lambda x: np.abs(x))    
    df_new = df_new.sort_values(by='abs', ascending=False)[[response]]    
    return df_new

In [86]:
scaler = RobustScaler()

In [88]:
df_combo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83487 entries, 1 to 89845
Data columns (total 24 columns):
Address            83486 non-null object
BldgArea           83487 non-null int64
BldgDepth          83487 non-null int64
BuiltFAR           83487 non-null int64
CommFAR            83487 non-null int64
FacilFAR           83487 non-null int64
Lot                83487 non-null int64
LotArea            83487 non-null int64
LotDepth           83487 non-null int64
NumBldgs           83487 non-null int64
NumFloors          83487 non-null int64
OfficeArea         83487 non-null int64
ResArea            83487 non-null int64
ResidFAR           83487 non-null int64
RetailArea         83487 non-null int64
YearBuilt          83487 non-null int64
YearAlter1         83487 non-null int64
ZipCode            83487 non-null int64
YCoord             83487 non-null int64
XCoord             83487 non-null int64
complaintCnt       83487 non-null int64
bldg_age           83487 non-null int64
bldg_alt_a

#### Let us move some fields out of feature list like YearBuilt, YearAlter1 which we are already taking care of as bld_age and bld_alt_age

In [96]:
feats = df_combo.columns.tolist()
response = 'complaint_range'
feats.remove(response)
feats.remove('complaintCnt')
feats.remove('Address')
feats.remove('YearBuilt')
feats.remove('YearAlter1')


X = df_combo[feats]
y = df_combo[response]

X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [97]:
response = 'complaint_range'
order_corr(df_combo,response).head(20)

,complaint_range
complaint_range,1.000
complaintCnt,0.553
NumFloors,0.342
BuiltFAR,0.223
BldgDepth,0.213
ResidFAR,0.211
FacilFAR,0.187
ResArea,0.123
BldgArea,0.105
YearAlter1,0.089


#### The above table shows that there are some corelation between complaint_range and some of the buiulding chracteristics like NumFloors, BuiltFAR, ResArea etc. Let us validate this with few more approaches.

#### Next let us try to find Feature Importance using Ensemble based techniques like Random Forest and XGBoost

In [102]:
clsfRF = RandomForestClassifier(n_estimators=100, random_state=5)
pipeRFC = Pipeline(steps=[('scaler', scaler), ('classification', clsfRF)])

In [103]:
pipeRFC.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('classification', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min...mators=100, n_jobs=None,
            oob_score=False, random_state=5, verbose=0, warm_start=False))])

In [104]:
model = pipeRFC.steps[1][1]
model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=5, verbose=0, warm_start=False)

In [105]:
total_importance = sum(model.feature_importances_)
col_names = X.columns.tolist()
feat_importance = model.feature_importances_

most_imp_ml_features = pd.DataFrame()
most_imp_ml_features['name'] = col_names
most_imp_ml_features['importance'] = feat_importance
most_imp_ml_features['percentage_importance'] = most_imp_ml_features['importance'].apply(lambda x: 1.0 * x / total_importance)

print(most_imp_ml_features.percentage_importance.sum())

1.0


In [106]:
most_imp_ml_features = most_imp_ml_features.sort_values(by='importance',ascending=False)
most_imp_ml_features[['name','percentage_importance']].head(20)

,name,percentage_importance
11,ResArea,0.142
16,XCoord,0.105
15,YCoord,0.104
5,Lot,0.095
0,BldgArea,0.091
6,LotArea,0.069
1,BldgDepth,0.067
9,NumFloors,0.052
18,bldg_alt_age,0.047
17,bldg_age,0.046


In [107]:
regXGBC = XGBRegressor(observation = 'multi:softmax')
pipeXGBC = Pipeline(steps=[('scaler', scaler), ('regression', regXGBC)])

In [108]:
pipeXGBC.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('regression', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_... reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1))])

In [109]:
modelXGB = pipeXGBC.steps[1][1]
modelXGB

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear',
       observation='multi:softmax', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [110]:
total_importanceXGB = sum(modelXGB.feature_importances_)
col_names = X.columns.tolist()
feat_importance = modelXGB.feature_importances_

most_imp_ml_features = pd.DataFrame()
most_imp_ml_features['name'] = col_names
most_imp_ml_features['importance'] = feat_importance
most_imp_ml_features['percentage_importance'] = most_imp_ml_features['importance'].apply(lambda x: 1.0 * x / total_importanceXGB)

print(most_imp_ml_features.percentage_importance.sum())

1.0


In [111]:
most_imp_ml_features = most_imp_ml_features.sort_values(by='importance',ascending=False)
most_imp_ml_features[['name','percentage_importance']].head(20)

,name,percentage_importance
11,ResArea,0.348
17,bldg_age,0.067
2,BuiltFAR,0.065
18,bldg_alt_age,0.062
7,LotDepth,0.062
1,BldgDepth,0.062
9,NumFloors,0.061
5,Lot,0.052
16,XCoord,0.052
15,YCoord,0.045


#### Both of Random Forest and XGBoost shows some relationship between the complaint_range and Building characteristics like ResArea, bldg_age, BuiltFar, etc.

## Concluding Remarks - Some of the building characteristics like Age of the Building (or Year Built), Residential Area, BuiltFar seems to have some relationship with Number of Heating Complaints as verified by two diffent algorithms.

In [112]:
df_combo.to_pickle('./DataFromNoteBookQ3.pkl')

In [67]:
client_4583a2827cdd4084aebc758e0b87cb12.upload_file('./DataFromNoteBookQ3.pkl',bucket,'DataFromNoteBookQ3_cos.pkl')

## Exaplanation of Response Score :

### This response is scored as 4. That is because the response is supported by proper modeling approach and performance evaluation. Also since this response does validate the result using another approach. However, since this response does not have any visualization it does not get a score of 5.